In [9]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday
from collections import Counter 

from dateutil.rrule import rrule, WEEKLY, SA, SU
from datetime import date

%matplotlib inline

In [10]:
#Task1, создание календаря

class RussianBusinessCalendar(AbstractHolidayCalendar):
    def __init__(self):
        ts = pd.read_csv('./data1.csv', usecols=['Год/Месяц','Январь','Февраль','Март','Апрель','Май','Июнь',
                                             'Июль','Август','Сентябрь','Октябрь','Ноябрь','Декабрь'],
                                             index_col=['Год/Месяц'], parse_dates=True)
        self.dates = pd.date_range(start=ts.index[0], end=(ts.index[ts.index.size-1] + pd.Timedelta('364D')), freq='D')
        
        self.start_date = self.dates[0]
        self.end_date = self.dates[self.dates.size-1]
        months = [ts.Январь, ts.Февраль, ts.Март, ts.Апрель, ts.Май, ts.Июнь, ts.Июль, ts.Август, ts.Сентябрь, ts.Октябрь, ts.Ноябрь, ts.Декабрь]
        
        self.Hw = []
        self.transf = []
        
        #Создаём список выходных дней, включая праздничные (Hw) и список перенесённых с праздников выходных (transf)
        for i in range (self.end_date.year-self.start_date.year+1):
            for j in range (12):
                d = [int(k) for k in months[j][i].replace('*','').replace('+','').split(',')]
                tmp = [datetime.datetime(self.start_date.year+i, j+1, d[z]) for z in range (len(d)-1)]
                self.Hw += tmp                 
                for k in range (len(months[j][i])-1):
                    if (months[j][i][k] == '*' or months[j][i][k] == '+'):
                        if (months[j][i][k-2] == ','):
                            self.transf.append(datetime.datetime(self.start_date.year+i, j+1, int(months[j][i][k-1])))
                            continue
                        elif (k == 1):
                            self.transf.append(datetime.datetime(self.start_date.year+i, j+1, int(months[j][i][k-1])))
                            continue
                        elif (1<=int(months[j][i][k-2]) and int(months[j][i][k-2])<=3):
                            self.transf.append(datetime.datetime(self.start_date.year+i, j+1, 10*int(months[j][i][k-2])+int(months[j][i][k-1])))
        
        #Найдём наиболее частотные выходные дни, и составим, используя transf (список переносов), список праздников
        self.Holidays = []
        Hw_s = pd.to_datetime(self.Hw).strftime('%d-%m').to_list()
        transf_s = pd.to_datetime(self.transf).strftime('%d-%m').to_list()
        freq, val = zip(*sorted(zip(Counter(Hw_s).values(), Counter(Hw_s).keys()), reverse = True))
        freq = list(freq)
        val = list(val)
        freq_max = np.where(np.array(freq) > 22, np.array(freq), 0)
        freq_max = freq_max[freq_max != 0]
        
        for i in range (freq_max.size-1):
            counter = transf_s.count(val[i])
            freq[i] -= counter
            
        freq, val = zip(*sorted(zip(freq, val), reverse=True))
        freq = list(freq)
        val = list(val)
        freq_max = np.where(np.array(freq) > 22, np.array(freq), 0)
        freq_max = freq_max[freq_max != 0]
        
        for i in range (freq_max.size-1):
            self.Holidays.append(val[i]) 
        self.Holidays = np.array(self.Holidays)
        self.rules = [(Holiday('Hw{}'.format(i), month=pd.to_datetime(self.Holidays[i]).month, 
                              day=pd.to_datetime(self.Holidays[i]).day) for i in range (len(self.Holidays)))]
                          
rusbusday = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri', calendar=RussianBusinessCalendar())
RBC = RussianBusinessCalendar()

In [11]:
#Task1, dataframe

L = len(RBC.dates)

C_1 = np.where((RBC.dates).isin(RBC.Hw), True, False) #является ли день выходным
C_21 = [C_1[i] and (RBC.dates[i].month == 1) and (RBC.dates[i].day<11) for i in range (L)]
C_22 = np.where((RBC.dates).isin(RBC.Holidays), True, False) #является ли день праздником
C_2 = [C_22[i] or C_21[i] for i in range (L)]
C_31 = np.where((RBC.dates+pd.offsets.Day(1)).isin(RBC.Holidays), True, False)
C_32 = np.where((RBC.dates).isin(RBC.Hw), False, True) #является ли день рабочим
C_3 = [C_31[i] and C_32[i] for i in range (L)]
C_41 = np.where(RBC.dates == (RBC.dates+pd.offsets.MonthEnd(1)), True, False)
C_4 = [C_41[i] and C_32[i] for i in range (L)]
C_n = np.where(RBC.dates + pd.offsets.YearEnd(1) <= RBC.dates + pd.offsets.Day(6), True, False) #последняя неделя года
for i in range (L):
    if (RBC.dates[i] + pd.offsets.YearEnd(1) > RBC.dates[i] + pd.offsets.Day(6)):
        continue
    else:
        C_n[i] =( (RBC.dates[i-1] == rrule(freq=WEEKLY, dtstart=RBC.dates[i-1], byweekday=SA, count=1)[0] and 
                  RBC.dates[i] == rrule(freq=WEEKLY, dtstart=RBC.dates[i], byweekday=SU, count=1)[0]) or
                  (RBC.dates[i] == rrule(freq=WEEKLY, dtstart=RBC.dates[i], byweekday=SA, count=1)[0]) and
                  (RBC.dates[i+1]>RBC.dates[i] + pd.offsets.YearEnd(1)))


df = pd.DataFrame({'выходной': C_1,
                   'праздник': C_2,
                   'предпраздничный рабочий день': C_3, 
                   'последний рабочий день месяца': C_4,
                   'пн': np.where(RBC.dates.weekday == 1, True, False),
                   'вт': np.where(RBC.dates.weekday == 2, True, False),
                   'ср': np.where(RBC.dates.weekday == 3, True, False),
                   'чт': np.where(RBC.dates.weekday == 4, True, False),
                   'пт': np.where(RBC.dates.weekday == 5, True, False),
                   'сб': np.where(RBC.dates.weekday == 6, True, False),
                   'вс': np.where(RBC.dates.weekday == 7, True, False),
                   'январь': np.where(RBC.dates.month == 1, True, False),
                   'февраль': np.where(RBC.dates.month == 2, True, False),
                   'март': np.where(RBC.dates.month == 3, True, False),
                   'апрель': np.where(RBC.dates.month == 4, True, False),
                   'май': np.where(RBC.dates.month == 5, True, False),
                   'июнь': np.where(RBC.dates.month == 6, True, False),
                   'июль': np.where(RBC.dates.month == 7, True, False),
                   'август': np.where(RBC.dates.month == 8, True, False),
                   'сентябрь': np.where(RBC.dates.month == 9, True, False),
                   'октябрь': np.where(RBC.dates.month == 10, True, False),
                   'ноябрь': np.where(RBC.dates.month == 11, True, False),
                   'декабрь': np.where(RBC.dates.month == 12, True, False),
                   'предновогодний выходной день': C_n },
                   index=RBC.dates)

df = df.astype(np.int8)
df


,выходной,праздник,предпраздничный рабочий день,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,...,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,предновогодний выходной день
1999-01-01,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-02,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-03,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1999-01-04,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-05,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2025-12-28,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
2025-12-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2025-12-30,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
#Task2.1
ts = pd.read_csv('./sber_stocks.csv', usecols=['CLOSE', 'TRADEDATE'], index_col=['TRADEDATE'], parse_dates=True)
ts = ts.CLOSE
df1 = pd.DataFrame({'исходно': ts.asfreq(rusbusday),
                   'лаг 1': ts.asfreq(rusbusday).shift(1, freq=rusbusday),
                   'лаг 3': ts.asfreq(rusbusday).shift(3, freq=rusbusday),
                   'лаг 5': ts.asfreq(rusbusday).shift(5, freq=rusbusday)})

print('Лаговые переменные рабочих дней с глубинами: 1, 3, 5')
df1

Лаговые переменные рабочих дней с глубинами: 1, 3, 5


,исходно,лаг 1,лаг 3,лаг 5
TRADEDATE,,,,
2013-03-25,98.79,NaN,NaN,NaN
2013-03-26,97.20,98.79,NaN,NaN
2013-03-27,96.75,97.20,NaN,NaN
2013-03-28,98.59,96.75,98.79,NaN
2013-03-29,98.76,98.59,97.20,NaN
...,...,...,...,...
2020-01-13,NaN,258.19,259.15,253.90
2020-01-14,NaN,NaN,257.99,NaN
2020-01-15,NaN,NaN,258.19,259.15


In [13]:
#Task2.2

ts = pd.DataFrame(ts)
ts['weekday'] = ts.index.weekday
df2 = pd.DataFrame({'mean 1': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).mean()),
                    'mean 3': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).mean()),
                    'mean 5': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).mean()),
                    'median 1': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).median()),
                    'median 3': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).median()),
                    'median 5': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).median()),
                    'max 1': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).max()),
                    'max 3': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).max()),
                    'max 5': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).max()),
                    'min 1': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).min()),
                    'min 3': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).min()),
                    'min 5': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).min()),
                    'std 1': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).std()),
                    'std 3': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).std()),
                    'std 5': ts.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).std())})

print('Скользящие статистики mean, median, max, min, std с окнами 1, 3, 5 сгрупированные по дням недели')
df2

Скользящие статистики mean, median, max, min, std с окнами 1, 3, 5 сгрупированные по дням недели


,mean 1,mean 3,mean 5,median 1,median 3,median 5,max 1,max 3,max 5,min 1,min 3,min 5,std 1,std 3,std 5
TRADEDATE,,,,,,,,,,,,,,,
2013-03-25,98.79,NaN,NaN,98.79,NaN,NaN,98.79,NaN,NaN,98.79,NaN,NaN,NaN,NaN,NaN
2013-03-26,97.20,NaN,NaN,97.20,NaN,NaN,97.20,NaN,NaN,97.20,NaN,NaN,NaN,NaN,NaN
2013-03-27,96.75,NaN,NaN,96.75,NaN,NaN,96.75,NaN,NaN,96.75,NaN,NaN,NaN,NaN,NaN
2013-03-28,98.59,NaN,NaN,98.59,NaN,NaN,98.59,NaN,NaN,98.59,NaN,NaN,NaN,NaN,NaN
2013-03-29,98.76,NaN,NaN,98.76,NaN,NaN,98.76,NaN,NaN,98.76,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-03,255.00,250.590000,245.624,255.00,252.06,244.71,255.00,255.00,255.00,255.00,244.71,235.14,NaN,5.300160,8.055503
2020-01-06,253.90,252.483333,247.188,253.90,253.90,248.80,253.90,254.75,254.75,253.90,248.80,237.04,NaN,3.218048,7.757897
2020-01-08,259.15,251.130000,245.060,259.15,248.04,246.20,259.15,259.15,259.15,259.15,246.20,231.59,NaN,7.006190,10.156483


In [15]:
#Task2.3

def bus_hol_bhol(ts):
    arr = np.where(ts.index.isin(RBC.Holidays), 1, 0)
    arr = np.where((ts.index+pd.offsets.Day(1)).isin(RBC.Holidays), 2, arr)
    return arr # 0 - рабочий, 1 - праздничный, 2 - предпраздничный

ts = pd.DataFrame(ts)
ts['bus_hol_bhol'] = bus_hol_bhol(ts)

df3 = pd.DataFrame({'mean': ts.groupby('bus_hol_bhol')['CLOSE'].transform(lambda x: x.expanding().sum()),
                    'median': ts.groupby('bus_hol_bhol')['CLOSE'].transform(lambda x: x.expanding().median()),
                    'max': ts.groupby('bus_hol_bhol')['CLOSE'].transform(lambda x: x.expanding().max()),
                    'min': ts.groupby('bus_hol_bhol')['CLOSE'].transform(lambda x: x.expanding().min()),
                    'std': ts.groupby('bus_hol_bhol')['CLOSE'].transform(lambda x: x.expanding().std())})

print('Статистики mean, median, max, min, std в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням')
df3

Статистики mean, median, max, min, std в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням


,mean,median,max,min,std
TRADEDATE,,,,,
2013-03-25,98.79,98.790,98.79,98.79,NaN
2013-03-26,195.99,97.995,98.79,97.20,1.124300
2013-03-27,292.74,97.200,98.79,96.75,1.071774
2013-03-28,391.33,97.895,98.79,96.75,1.010359
2013-03-29,490.09,98.590,98.79,96.75,0.968334
...,...,...,...,...,...
2020-01-03,249075.89,139.815,279.97,53.50,61.954344
2020-01-06,249329.79,139.930,279.97,53.50,61.991350
2020-01-08,249588.94,139.950,279.97,53.50,62.033702
